# Training data generation 

## importing libraries

In [1]:
import os
import os.path
import cv2
import glob
import imutils
import matplotlib.pyplot as plt
%matplotlib inline

## glob.glob used to find all the path names in the respective folder

In [2]:
image_files = glob.glob(os.path.join('/home/drake/Desktop/captcha scanner/solved-captchas/electoral-tagged', "*"))
OUTPUT_FOLDER='/home/drake/Desktop/captcha scanner/training'

### please refer to segmentation exmaple notebook for image numbers segmentation

In [3]:
for (i, captcha_file) in enumerate(image_files):             #iterating over image files stored in list
    filename = os.path.basename(captcha_file)               
    captcha_text = os.path.splitext(filename)[-2]
    #print(captcha_text)
    image = cv2.imread(captcha_file)                  #read the file
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)          #converting to grayscale
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]  # binarize in ivert color
    contours,_ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #finding contours
    image_regions = []                                              
    for contour in contours:                                  #drawing contours
        (x, y, w, h) = cv2.boundingRect(contour)
        if w / h > 1.25:
            half_width = int(w / 2)
            image_regions.append((x, y, half_width, h))
            image_regions.append((x + half_width, y, half_width, h))
        else:
            image_regions.append((x, y, w, h))
    image_regions = sorted(image_regions, key=lambda x: x[0])
    if len(image_regions) != len(captcha_text):
        continue
    i=0    #counter for iterating over the image label
    count=0  #counter for naming the images in their respective folder
    for letter_bounding_box in image_regions:                             #bounding boxes
        x,y,w,h = letter_bounding_box
        x,y,w,h= int(x), int(y), int(w), int(h)
        letter_image = gray[y - 2:y + h + 2, x - 2:x + w + 2]
        _,letter_image= cv2.threshold(letter_image,127,255,cv2.THRESH_BINARY)  #  again binarizing the image of individual numebers
        title = captcha_text[i]                       # determining the label of the number in the captcha
        # Get the folder to save the image in
        save_path = os.path.join(OUTPUT_FOLDER, title)
        # if the output directory does not exist, create it
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        os.chdir(save_path)
        count=len(os.listdir(save_path))   #checking the number of images in the folder to increment the iterater for labeling
        count=count+1
        count= str(count)+ '.jpg'
        cv2.imwrite(count, letter_image) #writing the file in the respective folder
        i=i+1
        
        

        
    
    